In [2]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import scipy.optimize as opt

from sklearn.linear_model import LogisticRegression, HuberRegressor

In [18]:
def func(x, a, b, c):
  # print(type(x[0]), type(a), type(b), type(c))
  return a * np.power(x, -b) + c

def func_rev(x, a, b, c):
  # print(type(x[0]), type(a), type(b), type(c))
  return a * np.power(x, b) + c


def get_best_fit(x, y, p0=[25.85993457, 0.15299458, 1.61171928], f=func_rev):
  xs, ys = zip(*sorted(zip(x, y)))
  xs = list(xs)
  ys = list(ys)
  optimizedParameters, pcov = opt.curve_fit(f, xs, ys, p0=p0);
  print(optimizedParameters)
  YS = [f(a, *optimizedParameters) for a in xs]
  # Use the optimized parameters to plot the best fit
  return xs, YS

In [21]:
def func2(X, a, b, c):
  # print(type(x[0]), type(a), type(b), type(c))
  N, D = X[0], X[1]
  return a * np.power(N * D, -b) + c # * np.power(D, -d) + e

def get_scaling_fit(x1, x2, y):
  x1s, x2s, ys = zip(*sorted(zip(x1, x2, y)))
  x1s = list(x1s)
  x2s = list(x2s)
  ys = list(ys)
  # optimizedParameters, pcov = opt.fmin_l_bfgs_b(func2, [x1s, x2s], ys) #p0=[10., -0.07,  1., -1., .1]) #[1, .1, 1, .1, 1]); # p0=[10, 0.1, 1, 1, 1]
  optimizedParameters, pcov = opt.curve_fit(func2, [x1s, x2s], ys, p0=[1., .2, 1.]) # ,p0=[14.0572608,   0.0967392,   1.,          1. ,         0.79611877]) # [10., .1,  1., 1., .1]) #[1, .1, 1, .1, 1]); # p0=[10, 0.1, 1, 1, 1]
  print(optimizedParameters)
  YS = [func2([x, y], *optimizedParameters) for x, y in zip(x1s, x2s)]
  # Use the optimized parameters to plot the best fit
  return x1s, x2s, YS, ys

In [15]:
def func23(X, a):
  # print(type(x[0]), type(a), type(b), type(c))
  p = [2.09176863e+02, 1.18897717e-01, 1.86816507e+00]
  N, D = X[0], X[1]
  return (p[0] * np.power( N * D * a, -p[1]) + p[2])  # * np.power(D, -d) + e

def get_scaling_fit2(x1, x2, y):
  x1s, x2s, ys = zip(*sorted(zip(x1, x2, y)))
  x1s = list(x1s)
  x2s = list(x2s)
  ys = list(ys)
  # optimizedParameters, pcov = opt.fmin_l_bfgs_b(func2, [x1s, x2s], ys) #p0=[10., -0.07,  1., -1., .1]) #[1, .1, 1, .1, 1]); # p0=[10, 0.1, 1, 1, 1]
  optimizedParameters, pcov = opt.curve_fit(func23, [x1s, x2s], ys) # ,p0=[14.0572608,   0.0967392,   1.,          1. ,         0.79611877]) # [10., .1,  1., 1., .1]) #[1, .1, 1, .1, 1]); # p0=[10, 0.1, 1, 1, 1]
  print(optimizedParameters)
  YS = [func23([x, y], *optimizedParameters) for x, y in zip(x1s, x2s)]
  # Use the optimized parameters to plot the best fit
  return x1s, x2s, YS, ys


In [23]:
DEFAULT_PROJECT_DIR = "/fsx-onellm/margaretli/env_srcs/xlf/xlformers_n/scaling/data"

col_names = ['C', 'D', 'N', 'lr', 'Avg Train Loss', 'Max Train Loss', 'C4 Eval PPL', 'Wiki Eval PPL', 'C4 Eval Loss', 'Wiki Eval Loss']

loss_name = "C4 Eval Loss"
csv_file = "/fsx-onellm/margaretli/env_srcs/xlf/xlformers_n/scaling/data/data.csv"
project_dir=DEFAULT_PROJECT_DIR

def read_data(csv_file, loss_name='C4 Eval Loss', col_names=col_names):
    mins_only = []
    df = pd.read_csv(csv_file, usecols=col_names,)
    df = df.loc[(df['lr'] >= 0)]
    n_vals = df['N'].unique()
    d_vals = sorted(df['D'].unique())
    for n in n_vals:
        for d in d_vals:
            cd_df = df[(df['N'] == n) & (df['D'] == d)]
            if cd_df.empty:
                continue
            min_index = cd_df['C4 Eval Loss'].idxmin()
            # print(min_index)
            # print(cd_df)
            # print(cd_df.loc[min_index])
            mins_only.append(cd_df.loc[min_index])

    mins_only_df = pd.DataFrame(mins_only)
    # print(mins_only_df.N.values[:5].astype(np.longdouble))
    # print(mins_only_df['N'].values[:5].astype(np.longdouble))
    # print(np.fromstring(mins_only_df.N.values[:5], dtype=np.longdouble))

    mins_only_df = mins_only_df[mins_only_df['N'] < 50000000]
    mins_only_df.dropna(subset=[loss_name], inplace=True)

    print(mins_only_df)
    # df.rename(columns={})
    return mins_only_df

data = read_data(csv_file, loss_name=loss_name, col_names=col_names)
x1s, x2s, Ys, ys = get_scaling_fit(data['N'], data['D'], data[loss_name])

for x1, x2, Y, y in zip(x1s, x2s, Ys, ys):
    print(f"{x1}, {x2}, {Y}, {y}")
# x1, x2, y = get_scaling_fit(pd.concat([data2[data2['Model Type'] == 'Baseline']['N(Non-Embedding Parameters)'],
#                                            data_iso[data_iso['Model Type'] == 'Baseline']['N(Non-Embedding Parameters)']]),
#                             pd.concat([data2[data2['Model Type'] == 'Baseline']['N(Tokens)'],
#                                            data_iso[data_iso['Model Type'] == 'Baseline']['N(Tokens)']]),
#                             pd.concat([data2[data2['Model Type'] == 'Baseline']['dev log ppl'],
#                                            data_iso[data_iso['Model Type'] == 'Baseline']['dev log ppl']]))

        lr  Avg Train Loss  Max Train Loss  C4 Eval PPL  Wiki Eval PPL  \
4    0.004           4.957           5.185      146.194        216.300   
12   0.004           4.717           5.044      119.259        166.031   
17   0.004           4.453           4.659       89.148        115.380   
21   0.004           3.999           4.218       61.665         70.939   
25   0.004           3.736           4.058       46.984         47.592   
29   0.004           3.533           3.764       39.858         35.852   
33   0.002           4.925           5.171      141.632        224.183   
37   0.002           4.683           5.024      114.945        168.517   
42   0.004           3.880           4.109       55.653         62.826   
46   0.004           3.603           3.968       41.767         39.030   
50   0.004           3.448           3.680       36.775         32.393   
54   0.004           3.384           3.679       34.108         29.042   
58   0.004           3.312           3